'''
Author:
        
        PARK, JunHo, junho@ccnets.org

        
        KIM, JoengYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.
'''

# Recyclable and Household Waste Classification 


https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification

In [1]:
import sys

path_append = "../" # Go up one directory from where you are.
sys.path.append(path_append) 

from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from nn.utils.init import set_random_seed
set_random_seed(0)

In [2]:
dataset_path = '../data/Recyclable and Household Waste Classification/images/images'

In [3]:
import os
import torch
import torch.nn.functional as F
import random
from PIL import Image
from torch.utils.data import Dataset


class WasteDataset(Dataset):
    def __init__(self, root_dir, split, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.image_paths = []
        self.labels = []
        
        for i, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for subfolder in ['default', 'real_world']:
                subfolder_dir = os.path.join(class_dir, subfolder)
                image_names = os.listdir(subfolder_dir)
                random.shuffle(image_names)
                
                if split == 'train':
                    image_names = image_names[:int(0.6 * len(image_names))]
                elif split == 'val':
                    image_names = image_names[int(0.6 * len(image_names)):int(0.8 * len(image_names))]
                else:  # split == 'test'
                    image_names = image_names[int(0.8 * len(image_names)):]
                
                for image_name in image_names:
                    self.image_paths.append(os.path.join(subfolder_dir, image_name))
                    self.labels.append(i)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label = torch.tensor(label, dtype=torch.long)
        label = F.one_hot(label, num_classes=30)
           
        return image, label

In [4]:
from torchvision import transforms

# Create the datasets and data loaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = WasteDataset(path_append + dataset_path, split='train', transform=transform)
val_dataset = WasteDataset(path_append + dataset_path, split='val', transform=transform)
test_dataset = WasteDataset(path_append + dataset_path, split='test', transform=transform)

X, y = train_dataset[0]
print(X.shape)
print(y.shape)

torch.Size([3, 224, 224])
torch.Size([30])


In [5]:

data_config = DataConfig(dataset_name = 'recycle_image', task_type='multi_label_classification', obs_shape=[3, 224, 224], label_size=30)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(core_model = 'resnet', encoder_model = 'none')
ml_params.model.core_config.d_model = 256

In [6]:
from trainer_hub import TrainerHub

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False)

In [7]:
trainer_hub.train(train_dataset, val_dataset)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Iterations:   0%|          | 0/140 [00:00<?, ?it/s]

c:\Users\ccnets-team\anaconda3\envs\torch\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


[0/100][50/140][Time 54.76]
Unified LR across all optimizers: 0.0001995308238189185
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0883	Gen: 1.7432	Rec: 1.7350	E: 0.0965	R: 0.0800	P: 3.3900
--------------------Test Metrics------------------------
accuracy: 0.3323

[0/100][100/140][Time 52.91]
Unified LR across all optimizers: 0.00019907191565870155
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.1228	Gen: 1.6427	Rec: 1.6092	E: 0.1563	R: 0.0892	P: 3.1291
--------------------Test Metrics------------------------
accuracy: 0.4137



Iterations:   0%|          | 0/140 [00:00<?, ?it/s]

...

[91/100][10/140][Time 36.29]
Unified LR across all optimizers: 0.00011117573150453981
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0314	Gen: 0.9516	Rec: 0.9544	E: 0.0286	R: 0.0341	P: 1.8746
--------------------Test Metrics------------------------
accuracy: 0.9763

[91/100][60/140][Time 36.36]
Unified LR across all optimizers: 0.00011092003441759838
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0311	Gen: 0.9516	Rec: 0.9549	E: 0.0278	R: 0.0343	P: 1.8754
--------------------Test Metrics------------------------
accuracy: 0.9699

[91/100][110/140][Time 36.27]
Unified LR across all optimizers: 0.00011066492541763764
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0334	Gen: 0.9497	Rec: 0.9527	E: 0.0303	R: 0.0364	P: 1.8690
--------------------Test Metrics------------------------
accuracy: 0.9737



Iterations:   0%|          | 0/140 [00:00<?, ?it/s]

[92/100][20/140][Time 36.27]
Unified LR across all optimizers: 0.00011041040315209514
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0319	Gen: 0.9514	Rec: 0.9544	E: 0.0289	R: 0.0349	P: 1.8739
--------------------Test Metrics------------------------
accuracy: 0.9745

[92/100][70/140][Time 36.22]
Unified LR across all optimizers: 0.00011015646627151911
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0310	Gen: 0.9504	Rec: 0.9535	E: 0.0279	R: 0.0341	P: 1.8728
--------------------Test Metrics------------------------
accuracy: 0.9814

[92/100][120/140][Time 36.27]
Unified LR across all optimizers: 0.0001099031134295615
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0322	Gen: 0.9516	Rec: 0.9546	E: 0.0292	R: 0.0352	P: 1.8740
--------------------Test Metrics------------------------
accuracy: 0.9730



Iterations:   0%|          | 0/140 [00:00<?, ?it/s]

[93/100][30/140][Time 36.37]
Unified LR across all optimizers: 0.00010965034328297077
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0332	Gen: 0.9508	Rec: 0.9539	E: 0.0300	R: 0.0363	P: 1.8715
--------------------Test Metrics------------------------
accuracy: 0.9815

[93/100][80/140][Time 37.88]
Unified LR across all optimizers: 0.00010939815449158482
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0310	Gen: 0.9500	Rec: 0.9530	E: 0.0280	R: 0.0340	P: 1.8720
--------------------Test Metrics------------------------
accuracy: 0.9730

[93/100][130/140][Time 38.13]
Unified LR across all optimizers: 0.00010914654571832365
--------------------Training Metrics--------------------
Trainer:  resnet
Inf: 0.0327	Gen: 0.9495	Rec: 0.9530	E: 0.0293	R: 0.0361	P: 1.8698
--------------------Test Metrics------------------------
accuracy: 0.9805

